In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [170]:
load("LR_database.rda")

In [171]:
LR_database$dup <- paste(LR_database$Receptor, LR_database$Ligand, sep="_")

In [172]:
subset_lr <- LR_database[LR_database$Pair.Name %in% LR_database$dup,  ] 

In [173]:
LR_database <- LR_database[!LR_database$Pair.Name %in% subset_lr$Pair.Name,  ] 

In [174]:
plexin_family <- as.vector(subset_lr[grep("plexin", subset_lr$Receptor.Name), ]$Receptor)

neuroligin_family <- as.vector(subset_lr[grep("neuroligin", subset_lr$Receptor.Name), ]$Receptor)

adam_family <- as.vector(subset_lr[grep("ADAM", subset_lr$Receptor.Name), ]$Receptor)

receptor_anno <- as.vector(subset_lr[grep("receptor", subset_lr$Ligand.Name), ]$Ligand)

In [175]:
ligand <- c("AGRN", "BMP2", "BMP4", "VTCN1", "CD244", "CD38", "GAS6", "GDNF", "GUCA2A", 
"HHLA2", "IHH", "PSEN1", "NLGN", "NRTN", "RPH3A", "SHH")

receptor <- c("CD2", "CD27", "CD80", "CD86", "SELL", "CD44", "CD81", "CD8A", "CLEC1B", 
"GLG1", "TYROBP")

In [176]:
ligand <- unique(c(ligand,plexin_family,neuroligin_family,adam_family))
receptor <- unique(c(receptor,receptor_anno))

In [177]:
subset_lr <- subset_lr[!subset_lr$Receptor %in% ligand,]

In [178]:
subset_lr <- subset_lr[!subset_lr$Ligand %in% receptor,]

In [179]:
dir <- subset_lr %>% filter(Pair.Name %in% subset_lr$dup & consensus_direction == 1)

In [180]:
subset_lr <- subset_lr[!subset_lr$Pair.Name %in% dir$Pair.Name,  ]
subset_lr <- subset_lr[!subset_lr$Pair.Name %in% dir$dup,  ]

In [181]:
no_dir <- subset_lr %>% filter(Pair.Name %in% subset_lr$dup & consensus_direction == 0)

In [182]:
subset_lr <- subset_lr[!subset_lr$Pair.Name %in% no_dir$Pair.Name,  ] 

In [183]:
df <- data.frame(matrix(ncol = ncol(subset_lr), nrow = 0))
x <- colnames(subset_lr)
colnames(df) <- x

In [184]:
for (lig in unique(no_dir$Ligand)) {
    vec_clean <- vector()
    
    r1 <- filter(no_dir, Ligand==lig | Receptor == lig)
    
    vec <- r1$Pair.Name
    
    for (component in vec) {
        comp <- unlist(strsplit(component, split = "_"))
        pair1 <- paste(comp[1], comp[2], sep = "_")
        pair2 <- paste(comp[2], comp[1], sep = "_")
        
        if (!pair1 %in% vec_clean & !pair2 %in% vec_clean) {
            vec_clean <- append(vec_clean, pair1)
        }
    }
    
    c1 <- r1[r1$Pair.Name %in% vec_clean,]
    
    df <- rbind(df,c1)
}

In [185]:
no_dir_df <- df[!duplicated(df$Pair.Name),]

In [186]:
subset_lr <- rbind(subset_lr,dir,no_dir_df)

In [187]:
LR_database <- rbind(subset_lr,LR_database)

In [188]:
rownames(LR_database) <- 1:nrow(LR_database) 

In [193]:
save(LR_database, file = "LR_database.rda")

In [195]:
write.csv(LR_database, "LR_database.csv")

In [196]:
getwd()

[1] "/home/mami/Maria/CommCompile/community_v3/community/data"